In [1]:
%load_ext dotenv
%dotenv

Python-dotenv could not parse statement starting at line 5


In [2]:
from llama_index import SimpleDirectoryReader, StorageContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_index.vector_stores import PGVectorStore

In [3]:
from decouple import config
import logging
import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

OPENAI_API_KEY = config('OPENAI_API_KEY')

In [6]:
import psycopg2

#initialize vectorstore config

PGVECTOR_CONNECTION_STRING = config('PGVECTOR_CONNECTION_STRING')
PGVECTOR_DATABASE = config('PGVECTOR_DATABASE')

connection_string = PGVECTOR_CONNECTION_STRING
db_name = PGVECTOR_DATABASE
conn = psycopg2.connect(connection_string)
conn.autocommit = True


In [7]:
from sqlalchemy.engine import make_url

# Load documents
documents = SimpleDirectoryReader("data").load_data()

url = make_url(connection_string)
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="DeloitteFutureOfAI",
    embed_dim=1536,  # openai embedding dimension
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, show_progress=True
)
query_engine = index.as_query_engine()

/Users/danielkwik/Library/CloudStorage/GoogleDrive-danielkwik21@gmail.com/My Drive/Coding/RAG_chatbot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 33/33 [00:01<00:00, 22.29it/s]


In [8]:
# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("How many AI-focused companies were there in 2018, and who reported that number?")
print(response)

There were over 2,000 AI-focused companies in the US alone in 2018. This number was reported by Statista.
